# 2.4 自动微分
求导是几乎所有深度学习优化算法的关键步骤。 虽然求导的计算很简单，只需要一些基本的微积分。 但对于复杂的模型，手工进行更新是一件很痛苦的事情（而且经常容易出错）。深度学习框架通过自动计算导数，即自动微分（automatic differentiation）来加快求导。 实际中，根据我们设计的模型，系统会构建一个计算图（computational graph）， 来跟踪计算是哪些数据通过哪些操作组合起来产生输出。 自动微分使系统能够随后反向传播梯度。 这里，反向传播（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。

## 2.4.1求导的几种情况
### 2.4.1.1 标量导数
![](./图片4-1标量导数.jpg)
### 2.4.1.2 亚导数
![](./图片4-2亚导数.jpg)
### 2.4.1.3 梯度：将导数拓展到向量
![](./图片4-3梯度.jpg)
### 2.4.1.4 拓展到矩阵
![](./图片4-4矩阵求导.jpg)

# 2.4.2 向量链式法则
![](./图片4-5向量链式法则.jpg)

**例1**：
![](./图片4-5向量链式法则例子1.jpg)

**例2**
![](./图片4-5向量链式法则例子2.jpg)

## 2.4.3 自动求导
自动求导计算一个函数在指定值上的导数，和符号求导、数值求导有一定区别。它依赖于计算图：
![](./图片4-6计算图.jpg)

自动求导的两种模式：正向累积与反向累积：
![](./图片4-7自动求导的两种模式.jpg)

一个例子：
![](./图片4-8自动求导的两种模式例子.jpg)

反向累积复杂度分析：
* 计算复杂度：O(n),n是操作子个数
    * 通常正向累计和反向累积的代价类似
* 内存复杂度：O(n)，因为要存储正向的所有中间结果

反向累积复杂度分析：
* 计算复杂度：O(n)计算一个变量的梯度
* 内存复杂度：O(1)


# 2.4.4 自动微分代码实现
## 一个简单的例子

作为一个演示例子，(**假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导**)。
首先，我们创建变量`x`并为其分配一个初始值。


In [22]:
import  torch
import warnings 
warnings.filterwarnings('ignore')
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

[**在我们计算$y$关于$\mathbf{x}$的梯度之前，我们需要一个地方来存储梯度。**]
重要的是，我们不会在每次对一个参数求导时都分配新的内存。
因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。
注意，一个标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状。


In [6]:
x.requires_grad_(True)  # 等价于 x = torch.arange(4.0, requires_grad=True)
x.grad

现在我们计算y:

In [7]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

`x`是一个长度为4的向量，计算`x`和`x`的点积，得到了我们赋值给`y`的标量输出。
接下来，我们[**通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度**]，并打印这些梯度。


In [9]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

[**现在让我们计算`x`的另一个函数。**]，注意在默认情况下，PyTorch会累积梯度，我们需要清除之前的值

In [10]:
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非标量变量的反向传播
当`y`不是标量时，向量`y`关于向量`x`的导数的最自然解释是一个矩阵。
对于高阶和高维的`y`和`x`，求导的结果可以是一个高阶张量。

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括[**深度学习中**]），
但当我们调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。
这里(**，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。**)

In [12]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定
# 微分函数关于self的梯度。在我们的例子中，我们只想求偏导数的和，
# 所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x # 对应位置相乘，y是一个向量
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## 分离计算
时，我们希望[**将某些计算移动到记录的计算图之外**]。
例如，假设`y`是作为`x`的函数计算的，而`z`则是作为`y`和`x`的函数计算的。
想象一下，我们想计算`z`关于`x`的梯度，但由于某种原因，我们希望将`y`视为一个常数，
并且只考虑到`x`在`y`被计算后发挥的作用。

在这里，我们可以分离`y`来返回一个新变量`u`，该变量与`y`具有相同的值，
但丢弃计算图中如何计算`y`的任何信息。
换句话说，梯度不会向后流经`u`到`x`。
因此，下面的反向传播函数计算`z=u*x`关于`x`的偏导数，同时将`u`作为常数处理，
而不是`z=x*x*x`关于`x`的偏导数。

In [13]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
x.grad

tensor([0., 1., 4., 9.])

In [14]:
x.grad.zero_()
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## Python控制流的梯度计算
使用自动微分的一个好处是：
[**即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度**]。
在下面的代码中，`while`循环的迭代次数和`if`语句的结果都取决于输入`a`的值。


In [15]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [21]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d / a

tensor(False)

## 小结
深度学习框架可以自动计算导数：首先将梯度附加到想要对其计算导数的变量上，然后记录目标函数的计算，然后执行它的反向传播过程，然后访问得到梯度。